In [2]:
#pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

In [35]:
import cv2
import torch
from PIL import Image
import os
import pandas as pd


In [49]:
img_directory = '/Users/sleung2/Documents/MIDS Program/Capstone_local/snapshot_wisconsin/all/yolo_splits4.1/test/images/'

# Model
model = torch.hub.load('ultralytics/yolov5', 'custom', path='yolov5s_best_serengeti_splits4.pt')#, source='local')

#Images
imgs = []
img_names = []

for img_name in os.listdir(img_directory):
    img = cv2.imread(img_directory+img_name)[:, :, ::-1]
    imgs.append(img)
    img_names.append(img_name)

    
# Inference
results = model(imgs, size=329)  # includes NMS



Using cache found in /Users/sleung2/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-11-27 torch 1.10.0 CPU

Fusing layers... 
Model Summary: 224 layers, 7078183 parameters, 0 gradients
Adding AutoShape... 
/Users/sleung2/opt/anaconda3/lib/python3.8/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [59]:
results.pandas()

In [57]:
i = 0

for df,image_name  in zip(results.pandas().xyxy, img_names):
    print(i)
    int_results_df = results.pandas().xyxy[i]
    int_results_df['image_name'] = image_name
    
    if i == 0:
        full_results_df = int_results_df
        
    else:
        full_results_df = pd.concat([full_results_df, 
                                    int_results_df])
        
    i+=1

0
1
2


KeyboardInterrupt: 

In [47]:
def convert_yolo_bbox(size, box):
    dw = 1./size[0]
    dh = 1./size[1]
    x = (box[0] + box[1])/2.0
    y = (box[2] + box[3])/2.0
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

# im=Image.open(img_path)
# w= int(im.size[0])
# h= int(im.size[1])


xmin = 31.410490
xmax = 151.594894
ymin = 178.841827
ymax = 328.401520 #define your x,y coordinates
b = (xmin, xmax, ymin, ymax)
bb = convert((329,329), b)

In [48]:
bb

(0.2781236838905775,
 0.7708865455927051,
 0.3653021398176292,
 0.45458873252279636)